In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

social = pd.read_csv("Aggregated_reddit_twitter.csv")
social = social.rename(columns={"date": "timestamp"})
social["timestamp"] = pd.to_datetime(social["timestamp"])

/tmp/ipykernel_194308/4215461173.py:5: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  social = pd.read_csv("Aggregated_reddit_twitter.csv")


In [2]:
social = social[social.content != "u/circular360"]
social = social[social.content != "There you go"]
social = social[
    social.content != "Why force the other player to stay around? Just give the winning player the option to continue the game with a perma passing AI controlling the surrendering player."]
# crypto = pd.read_csv("transformed_crypto.csv")
# crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)
crypto = pd.read_csv("crypto_differences.csv")
crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)

social = social.sort_values(by="timestamp")
# social.dtypes
sr = social.resample("T", on="timestamp").size()
freq = sr.reset_index()

freq = freq.rename(columns={0: "n_comments"})

In [3]:
freq["hour"] = freq["timestamp"].dt.hour
freq["day"] = freq["timestamp"].dt.dayofweek
freq["month"] = freq["timestamp"].dt.month
freq

,timestamp,n_comments,hour,day,month
0,2021-03-01 00:00:00,18,0,0,3
1,2021-03-01 00:01:00,8,0,0,3
2,2021-03-01 00:02:00,4,0,0,3
3,2021-03-01 00:03:00,11,0,0,3
4,2021-03-01 00:04:00,4,0,0,3
...,...,...,...,...,...
1132079,2023-04-26 03:59:00,0,3,2,4
1132080,2023-04-26 04:00:00,0,4,2,4
1132081,2023-04-26 04:01:00,0,4,2,4
1132082,2023-04-26 04:02:00,0,4,2,4


In [4]:
alp = pd.read_csv("df_alphavantage_scores.csv")
alp["timestamp"] = pd.to_datetime(alp["timestamp"])
alp = alp.drop("Unnamed: 0", axis=1)
print(alp.shape)

# alp = alp.fillna(0)
frqn = pd.merge(freq, alp, on="timestamp", how="left")
frqn = frqn.dropna(how="all", subset=["overall_sentiment_score", "BTC_relevance", "BTC_sentiment", "ETH_relevance",
                                      "ETH_sentiment"])
frqn = frqn.fillna(0)
frqn

(25466, 6)


,timestamp,n_comments,hour,day,month,overall_sentiment_score,BTC_sentiment,ETH_sentiment,BTC_relevance,ETH_relevance
526378,2022-03-01 12:58:00,0,12,1,3,-0.138797,-0.118125,0.000000,0.018982,0.000000
527070,2022-03-02 00:30:00,0,0,2,3,0.021230,0.000000,0.000892,0.000000,0.053967
527192,2022-03-02 02:32:00,0,2,2,3,-0.031243,0.006883,0.000000,0.063967,0.000000
527520,2022-03-02 08:00:00,0,8,2,3,-0.174572,-0.009523,0.000000,0.125701,0.000000
527640,2022-03-02 10:00:00,0,10,2,3,-0.036470,0.001776,0.001233,0.162677,0.023400
...,...,...,...,...,...,...,...,...,...,...
1012170,2023-02-01 21:30:00,0,21,2,2,0.089214,0.283323,0.000000,0.630142,0.000000
1012203,2023-02-01 22:03:00,0,22,2,2,0.038911,-0.091982,0.000000,0.046139,0.000000
1012230,2023-02-01 22:30:00,0,22,2,2,0.162278,0.058989,0.000000,0.056275,0.000000
1012259,2023-02-01 22:59:00,0,22,2,2,-0.033137,0.000000,-0.083125,0.000000,0.185306


In [5]:
mg = pd.merge(crypto, frqn, on="timestamp", how="inner")

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

mg.shape

(15183, 16)

In [7]:
shortmg = mg.loc[0:]
shortmg.dropna()


,Unnamed: 0,XRP-PERP,SOL-PERP,ETH-PERP,DOGE-PERP,BTC-PERP,timestamp,n_comments,hour,day,month,overall_sentiment_score,BTC_sentiment,ETH_sentiment,BTC_relevance,ETH_relevance
0,611127,0.030873,0.003136,-0.003228,0.000491,0.008013,2022-03-01 12:58:00,0,12,1,3,-0.138797,-0.118125,0.000000,0.018982,0.000000
1,611819,0.000000,-0.001285,0.006197,-0.000774,0.028552,2022-03-02 00:30:00,0,0,2,3,0.021230,0.000000,0.000892,0.000000,0.053967
2,611941,0.004518,0.008197,0.001328,0.000801,0.006267,2022-03-02 02:32:00,0,2,2,3,-0.031243,0.006883,0.000000,0.063967,0.000000
3,612269,0.012173,0.002481,0.000525,-0.006094,0.014352,2022-03-02 08:00:00,0,8,2,3,-0.174572,-0.009523,0.000000,0.125701,0.000000
4,612389,-0.002718,-0.002229,0.000740,0.001686,-0.002714,2022-03-02 10:00:00,0,10,2,3,-0.036470,0.001776,0.001233,0.162677,0.023400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15178,875067,0.000451,0.000326,0.000977,-0.000086,0.000418,2022-08-31 20:41:00,0,20,2,8,0.026801,-0.334158,0.000000,0.235823,0.000000
15179,875071,-0.001382,0.000054,-0.000488,-0.000031,0.000070,2022-08-31 20:45:00,0,20,2,8,0.188588,0.000000,0.029589,0.000000,0.350564
15180,875094,-0.000662,-0.000571,-0.000490,-0.000508,-0.000465,2022-08-31 21:08:00,0,21,2,8,0.154257,0.221177,0.000000,0.803061,0.000000
15181,875118,0.000090,0.000981,0.000692,-0.000022,0.000978,2022-08-31 21:32:00,0,21,2,8,-0.009736,-0.256727,0.350953,0.733479,0.421485


In [8]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)


class MyTimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data.loc[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)

# data = MyTimeSeriesDataset(X_data=shortmg[['n_comments', 'XRP-PERP', 'ETH-PERP']], y_data=shortmg['BTC-PERP'])

In [9]:
alp.columns

Index(['overall_sentiment_score', 'BTC_sentiment', 'ETH_sentiment',
       'BTC_relevance', 'ETH_relevance', 'timestamp'],
      dtype='object')

In [79]:
# Load your dataset into a pandas DataFrame called 'data'
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Scale the features using MinMaxScaler
cs = ['BTC-PERP', 'XRP-PERP', 'ETH-PERP', 'SOL-PERP', 'DOGE-PERP', 'overall_sentiment_score', 'BTC_sentiment',
      'ETH_sentiment', 'BTC_relevance', 'ETH_relevance']
scaled_data = shortmg[cs + ["hour", "day", "month", "n_comments"]]
# scaled_data = shortmg[['XRP-PERP', 'ETH-PERP', 'BTC-PERP', 'SOL-PERP', 'DOGE-PERP', 'n_comments']].to_numpy()

# scaled_data = abs(scaled_data)


# scaled_data = scaled_data.to_numpy()
#
scaler = StandardScaler()
#
scaled_data = scaled_data.to_numpy()
# scaled_data = scaler.fit_transform(scaled_data)


# Create sequences and targets for the RNN model
seq_length = 60
X_data = []
y_data = []

future = 5

for i in range(len(scaled_data) - seq_length - future):
    print(f"{i} / {len(scaled_data) - seq_length - future}")
    # if i > 100:
    #     break
    X_data.append(scaled_data[i:i + seq_length].flatten())
    # y_data.append(sum(crypto["timestamp"].between_time(
    #     shortmg["timestamp"][i + seq_length], shortmg["timestamp"][i + seq_length + future])
    # ))
    # print(crypto.dtypes, shortmg.dtypes)
    mask = (crypto["timestamp"] > shortmg["timestamp"][i + seq_length]) & (
                crypto["timestamp"] < shortmg["timestamp"][i + seq_length + future])
    x = crypto[mask]
    y_data.append(sum(x["BTC-PERP"]))

x_data = scaler.fit_transform(X_data)
# y_data = np.delete(y_data, [-1, -2, -3, -4], axis=1)
# y_data = np.delete(y_data, [-1, -2, -3, -4, -5, -6, -7, -8], axis=1)
# y_data = np.array(y_data).T[0].T
y_data = np.array(y_data)
# to make classifier
epsilon = 0.0


def f(x):
    if x >= epsilon:
        return 1
    elif x < -epsilon:
        return 2
    else:
        return 0

# print(y_data)

y_data = list(map(f, y_data))

print(y_data[1:10])
X_data, y_data = np.array(X_data), np.array(y_data)

y_data = y_data.astype(np.int64)


0 / 15118
1 / 15118
2 / 15118
3 / 15118
4 / 15118
5 / 15118
6 / 15118
7 / 15118
8 / 15118
9 / 15118
10 / 15118
11 / 15118
12 / 15118
13 / 15118
14 / 15118
15 / 15118
16 / 15118
17 / 15118
18 / 15118
19 / 15118
20 / 15118
21 / 15118
22 / 15118
23 / 15118
24 / 15118
25 / 15118
26 / 15118
27 / 15118
28 / 15118
29 / 15118
30 / 15118
31 / 15118
32 / 15118
33 / 15118
34 / 15118
35 / 15118
36 / 15118
37 / 15118
38 / 15118
39 / 15118
40 / 15118
41 / 15118
42 / 15118
43 / 15118
44 / 15118
45 / 15118
46 / 15118
47 / 15118
48 / 15118
49 / 15118
50 / 15118
51 / 15118
52 / 15118
53 / 15118
54 / 15118
55 / 15118
56 / 15118
57 / 15118
58 / 15118
59 / 15118
60 / 15118
61 / 15118
62 / 15118
63 / 15118
64 / 15118
65 / 15118
66 / 15118
67 / 15118
68 / 15118
69 / 15118
70 / 15118
71 / 15118
72 / 15118
73 / 15118
74 / 15118
75 / 15118
76 / 15118
77 / 15118
78 / 15118
79 / 15118
80 / 15118
81 / 15118
82 / 15118
83 / 15118
84 / 15118
85 / 15118
86 / 15118
87 / 15118
88 / 15118
89 / 15118
90 / 15118
91 / 1511

In [80]:
train_size = int(len(X_data) * 0.8)
X_train, X_test = X_data[:train_size], X_data[train_size:]
y_train, y_test = y_data[:train_size], y_data[train_size:]

In [81]:
from collections import Counter

Counter(y_train), Counter(y_test)

(Counter({2: 5927, 1: 6167}), Counter({2: 1481, 1: 1543}))

In [82]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, balanced_accuracy_score, accuracy_score, confusion_matrix

# regressor = RandomForestRegressor(random_state=0, max_depth=5, n_estimators=20)
# regressor = SVR()
regressor = LogisticRegression()
regressor.fit(X_train, y_train)
pred = regressor.predict(X_test)

balanced_accuracy_score(pred, y_test), accuracy_score(pred, y_test)

/home/dario/PycharmProjects/BAINSACryptocurrencies/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


(0.5285014781966002, 0.5228174603174603)

In [87]:
confusion_matrix(y_test, pred)

array([[ 578,  965],
       [ 478, 1003]])

In [49]:
from torch.utils.data import DataLoader

batch_size = 256
train_dataset = TimeSeriesDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
test_dataset = TimeSeriesDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [50]:
# classifier
# y_data = y_data.flatten()
# y_data.shape

In [51]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, output_size):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Linear(input_size, d_model)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers)
        self.fc = nn.Linear(d_model, output_size)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        x = self.fc(x[:, -1, :])
        return x


class DenseModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.lin = nn.Linear(input_size, 256)
        self.ff = nn.Sequential(
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, output_size)
        )

    def forward(self, x):
        # x = torch.flatten(x, 1)
        x = self.lin(x)
        x = self.ff(x)
        o = f.log_softmax(x, dim=1)
        return o

In [52]:
from torch import optim
import torch.nn.functional as f

input_size = X_data.shape[1]
# hidden_size = 256
# num_layers = 8
# output_size = y_data.shape[1]
output_size = 3
# output_size = 1

# model = RNNModel(input_size, hidden_size, num_layers, output_size)
# model = TransformerModel(input_size, d_model=32, nhead=2, num_layers=2, output_size=output_size)
model = DenseModel(input_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
input_size, output_size

(840, 3)

In [53]:
import platform

num_epochs = 30
log = False
#
if "Linux" in platform.platform():
    model = torch.compile(model)
#
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)
        #
        optimizer.zero_grad()
        y_batch = y_batch.type(torch.LongTensor)
        loss = f.nll_loss(y_pred, y_batch)

        loss.backward()
        optimizer.step()
        # optimizer.step()

        if batch_idx % 20 == 0 and log:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(X_batch), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss))

    model.eval()
    with torch.no_grad():
        correct = 0
        test_loss = 0
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            y_batch = y_batch.type(torch.LongTensor)

            loss = f.nll_loss(y_pred, y_batch)
            test_loss += loss

            pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(y_batch.view_as(pred)).sum().item()

        test_loss = test_loss / len(test_loader)

        print(
            f"Epoch: {epoch + 1}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss:.4f}, accuracy: {100 * correct / len(test_loader.dataset)}")



Epoch: 1, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 2, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 3, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 4, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 5, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 6, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 7, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 8, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 9, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 10, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 11, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 12, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.97520661157025
Epoch: 13, Train Loss: 0.7142, Test Loss: 0.7147, accuracy: 46.9752066115

KeyboardInterrupt: 